In [1]:
#Import Modules
import numpy as np
import scipy.io as io
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from datetime import date, timedelta
from datetime import datetime
from scipy import io as io
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import pygrib
import urllib
import ftplib
import os
import xarray as xr
import pandas as pd

from cartopy import crs as ccrs, feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.util as cutil

#PREDEFINED VARIABLES
monthi = 12 #Month used for calculations

#PATHS


nx=179
ny=89
ngrids=nx*ny #16380 grid size
nleads=16 #Total number of leads
nretained=200 #Total number of retained EOF's
#v = np.zeros((ngrids,200, nleads,)) #This empty array will hold all spatial eigenvectors according to lead for each month
#w = np.zeros((ngrids,nleads)) #This empty array will hold all spatial eigenvalues according to lead for each month
W = np.zeros((ngrids,nleads))
V = np.zeros((ngrids,nretained, nleads))
climo_days = 382 #Used to define axis 1 of gefsclimo and ecclimo arrays. Accounts for 366 days of a year, and 16 additional forecast days

#Empty arrays to store eigenvecotrs 
I80s = np.zeros((nleads)) #This empty array will be filled with the number of EOFs that explain 80% of variance according to lead for each month
I80s_cum = np.zeros(nleads+1) #Create empty array that will store the cumulative sum of the retained EOFs for all leads

#Create Date Index 
#This Date Index will be used to remove the Seasonal Cycle at different lead times
d0=date(2000,1,1)
d1=date(2019,12,31)
L=(d1-d0).days
datelist=np.array([d0+timedelta(days=int(x)) for x in np.arange((L))])
monthlist=np.array([d.month for d in datelist])
yearlist=np.array([d.year for d in datelist])
daylist=np.array([d.day for d in datelist])
Ndaysinmonthi15=np.sum(np.logical_and(monthlist==monthi,yearlist<2015))
Ndaysinmonthi20=np.sum(np.logical_and(monthlist==monthi,yearlist>=2015))

#Create list of dates that will be used for indexing in creation of gefs climatology of Z200
doylist=np.array([(datelist[d]-date(datelist[d].year-1,12,31)).days for d in np.arange(datelist.shape[0])])
I = np.where(doylist == 366)[0]
doylist[I] = 365

#Xarray Conversion(s)
time = np.arange(365)
lats = np.arange(-89,89,2) #Grid spacing will need to be redefinied to fit CPC grid spacing
lons = np.arange(0,358,2)
lead = np.arange(nleads)

#Spacial Definitions
latS = -88
latN = 88
lonW = 0
lonE = 356
clons = (lonW+lonE)/2.

#Define the grid
lons = np.arange(0,358, 2)
lats = np.arange(-89,89, 2)
dx = np.abs((lons[2]-lons[0]))
dy = np.abs((lats[2]-lats[0]))
dx, dy
lonRange = np.arange(lonW, lonE + dx, dx)
latRange = np.arange(latS, latN + dy, dy)
proj_map = ccrs.PlateCarree(central_longitude=0)

#START OF CALCULATIONS

#Calculate the Seasonal Cycle, X, for gefs at each date in the above Date Index
t=np.arange(L)
nharm=4
X=np.ones((L,2*nharm+1))
for n in np.arange(nharm):
     X[:,(n+1)*2-1]=np.sin(2*np.pi*(n+1)*t/365.25)
     X[:,(n+1)*2]=np.cos(2*np.pi*(n+1)*t/365.25)

#REFORECAST    
#define variables to create gefs_data array shape. For 1 degree grids, change latitude and longitude accordingly. Change forecast lead to modify temporal domain
latitude = 89 #89 gridpoints of latitude in 2x2 degree grid
longitude = 179 #179 gridpoints of longitude in 2x2 degree grid
#forecast_lead = 16 #16 days of forecast lead

#IMPORT REFORECAST DATA for each consecutive date, for a given lead. Data is 200 hPa Geopotential Height (Z200) given in meters

gefs_data=np.zeros(((d1-d0).days,latitude,longitude))
print('empty gefs_data array created with shape: '+str(gefs_data.shape))

#REANALYSIS
#IMPORT REANALYSIS DATA. Convert to a numpy file as needed. Make sure that the latitude and longitude grid is the SAME as gefs_data
era5data = np.load('/free2/sm957448/will_files/200mb_Z/era5_reanalysis/updated_era5data.npy')/10 #Divide by 10 to convert from meters to decameters
era5data = era5data[:7320,:,:]

#for month in np.arange(1,12,1): #Loop over all 12 months

def gefs_loader(lead):
    for d in np.arange(L): #Looping indexed days from 2000-2019 according to lead number
        if daylist[d] == 1: #Making sure that data loading begins only when d=1 to save memory
            if monthlist[d] < 10:
                if lead < 10:
                    a = np.load('/free2/sm957448/will_files/200mb_Z/gefs_reforecast/d1-10/gefs_200mbZ_110_0'+str(monthlist[d])+'.npy')
                else:
                    a = np.load('/free2/sm957448/will_files/200mb_Z/gefs_reforecast/d10-16/gefs_200mbZ_1016_0'+str(monthlist[d])+'.npy') 
            else: 
                if lead<10:
                    a = np.load('/free2/sm957448/will_files/200mb_Z/gefs_reforecast/d1-10/gefs_200mbZ_110_'+str(monthlist[d])+'.npy') 
                else:
                    a = np.load('/free2/sm957448/will_files/200mb_Z/gefs_reforecast/d10-16/gefs_200mbZ_1016_'+str(monthlist[d])+'.npy') 
                    
        if lead < 10:
            gefs_data[d,:,:] = a[(yearlist[d]-2000),daylist[d]-1,::-1,:,lead]
        else:
            gefs_data[d,:,:] = a[(yearlist[d]-2000),daylist[d]-1,::-1,:,lead-10]
    return gefs_data/10 #Divid by ten to make decameters.
            

#Create the GEFS Climatology of the Model Variable, with the seasonal cycle removed
def climmake(gefs_data):
    gefsclimo = np.zeros((climo_days,gefs_data.shape[1],gefs_data.shape[2]))
    for lead in np.arange(nleads):
        C=np.linalg.inv(X[Iinc,:].T.dot(X[Iinc,:])).dot(X[Iinc,:].T.dot(gefs_data[Iinc,:,:].transpose(1,0,2)).transpose(1,0,2))
        gefsclimo[:,:,:] = X.dot(C.transpose(1,0,2))[:gefsclimo.shape[0],:gefsclimo.shape[1],:]
    return gefsclimo
print('gefs climatology created')

for lead in np.arange(nleads):            
    #Perform Quality control on gefs_data by indexing days with missing data across all days and leads
    print('working on lead number ' +str(lead))
    #gefs_data=gefs_loader(lead) #Function which loads and creates gefs_data files as a .npy
    #np.save('/free2/sm957448/will_files/200mb_Z/gefs_data/gefs_data_lead'+str(lead)+'.npy', gefs_data)
    gefs_data=np.load('/free2/sm957448/will_files/200mb_Z/gefs_data/gefs_data_lead'+str(lead)+'.npy') #Loads saved gefs_data files as a .npy
    #print(gefs_data)
    print('gefs_data shape')
    print(gefs_data.shape)
    A=np.sum(gefs_data==0,axis=2)
    A=np.sum(A>0,axis=1)
    Iexc = np.where(A>0)[0] #Index of days that are excluded because they have missing data
    Iinc = np.where(A==0)[0] #Index of days that are included, no missing data
    
    #Calculate the seasonal cycle, X, for each date
    t=np.arange(L)
    nharm=4 #4 harmonics of the seasonal cycle will be removed
    X=np.ones((L,2*nharm+1)) #X represents the seasonal cycle of data
    for n in np.arange(nharm):
        X[:,(n+1)*2-1]=np.sin(2*np.pi*(n+1)*t/365.25) 
        X[:,(n+1)*2]=np.cos(2*np.pi*(n+1)*t/365.25)

    #Create gefs error anomalies by subtracting the climatology
    gefsanoms = np.zeros((gefs_data.shape))
    #gefsclimo=climmake(gefs_data)
    #np.save('/free2/sm957448/will_files/200mb_Z/gefs_climo/gefs_climo_lead'+str(lead)+'.npy', gefsclimo)
    gefsclimo=np.load('/free2/sm957448/will_files/200mb_Z/gefs_climo/gefs_climo_lead'+str(lead)+'.npy') #Loads saved gefs_climo files as a .npy
    
    gefsanoms[Iinc,:,:] = (((gefs_data[Iinc,:,:]-gefsclimo[doylist[Iinc],:,:])))
    print('gefsanoms created')

    #Adjust date objects to reflect the additional 16 days of forecast lead
    d0=date(2000,1,1)
    d1=date(2020,1,16)
    L=(d1-d0).days
    datelist=np.array([d0+timedelta(days=int(x)) for x in np.arange((L))])
    monthlist=np.array([d.month for d in datelist])
    yearlist=np.array([d.year for d in datelist])
    daylist=np.array([d.day for d in datelist])

    #Create list of dates that will be used for indexing in creation of era5 anomalies
    doylist=np.array([(datelist[d]-date(datelist[d].year-1,12,31)).days for d in np.arange(datelist.shape[0])])
    I = np.where(doylist == 366)[0]
    doylist[I] = 365

    #Create the ERA5 Climatology of the Model Variable, with the seasonal cycle removed
    era5anoms = np.zeros((era5data.shape)) #Import will need to be recalulated in order to fit CPC CFS data set, make sure the shapes of gefs_data and CFS match the appropriate arrays
    if lead == 0:
        #ecclimo=climmake(era5data)
        #np.save('/free2/sm957448/will_files/200mb_Z/ecclimo/ecclimo.npy', ecclimo)
        ecclimo=np.load('/free2/sm957448/will_files/200mb_Z/ecclimo/ecclimo.npy')
        print('ERA5 climatology created')
        #Create ERA5 anomalies by subtracting the climatology
        era5anoms[Iinc,:,:] = (era5data[Iinc,:,:]-ecclimo[doylist[Iinc],:,:])
        np.save('/free2/sm957448/will_files/200mb_Z/era5anoms/era5anoms.npy',era5anoms)
    else:
        era5anoms=np.load('/free2/sm957448/will_files/200mb_Z/era5anoms/era5anoms.npy')
        
    print('ERA5 anomalies created, begin calculating seasonal cycle of error')
    #CALCULATE AND PLOT MODEL SEASONAL CYCLE OF ERROR

    #Dispose of climatology arrays to save memory
    ecclimo = []
    gefsclimo = []

    #CALCULATE TOTAL ERROR ANOMALIES
    #Subtract the era5 anomalies from gefsanomalies according to lead 
    erroranoms=np.zeros((era5data.shape[0],gefs_data.shape[1], gefs_data.shape[2]))
    #for lead in np.arange(nleads): #Fill the array
    erroranoms[Iinc,:,:] = (((gefsanoms[Iinc,:,:]-era5anoms[Iinc+lead,:,:])))
    print('Total Error Anomalies Created. Beginning Spatial Eigenvector Calculations')

    #Dispose of unnecessary arrays to save memory
    #era5data = []
    era5anoms = []

    #NOW WE BEGIN THE STATISTICAL POSTPROCESSING ALGORITHM
    #Reset the Date Indexing one final time to return to the original 7304 day temporal domain, which erroranoms and gefsanoms abide by
    d0=date(2000,1,1)
    d1=date(2019,12,31)
    L=(d1-d0).days
    datelist=np.array([d0+timedelta(days=int(x)) for x in np.arange((L))])
    monthlist=np.array([d.month for d in datelist])
    yearlist=np.array([d.year for d in datelist])
    daylist=np.array([d.day for d in datelist])

    #Create monthly indexing to run the two-step EOF algorithm according to each month 
    #for month in np.arange(11,12,1):
    Imonths = np.where(monthlist==monthi)[0] #Create Indexing for the days in each month over the 20 year period, with all dates that contain missing data removed
    print(monthi)
    Xmonths = np.zeros((Imonths.shape[0],latitude*longitude)) #This empty array will be used to store spatial GEFSv12 data based on total error anomaly (erroranoms) data
    Xmonths_a = np.zeros((Imonths.shape[0],latitude*longitude)) #This empty array will be used store GEFSv12 anomaly (gefsanoms) data. The "a" stands for approximate, and will be used to create the reconstructions
    I15 = np.where(yearlist[Imonths] == 2015)[0][0] #I15 represents the index number of 1/1/15, when the training set turns into the testing set
    Inm_reindex = np.where(np.logical_and(yearlist[Imonths]<2015, gefsanoms[Imonths,10,10] != 0))[0] #Inm_reindex reindexes the dates of the training set to exclude missing dates, different from Iinc because it solely is for the training set.
    Imonths15 = Imonths[:I15] #Imonths15 is the index of dates of the selected months from 1/1/00 - 1/1/15
    Imonths20 = Imonths[I15:] #Imonths20 is the index of dates for the selected months from 1/1/15-12/31/19
    Xmonths15 = Xmonths[:I15,:] #Xmonths15 is the data matrix of the geopotential height erroranomalies up to 1/1/15
    Xmonths20 = Xmonths[I15:,:] #Xmonths20 is the data matrix of geopotential height error anomalies from 1/1/15-12/31/19
    Xmonths20_a = Xmonths_a[I15:,:] #Xmonths20_a is the data matrix of reconstructed geopotential height error anomalies from 1/1/15 - 12/31/19

        #lats = np.arange(-89,89,2) #Define the 2 degree latitude range. Used to help account for curvature of earth at high latitudes
#    w_spec = np.zeros((Xmonths15.shape[1],gefs_data.shape[3])) #This empty array will create the normalized eigenvalues (may be commented out)
    
    #Create Spatial EOFs
    #for lead in np.arange(nleads):
    for y in np.arange(latitude):
        Xmonths15[:,(y*longitude):((y+1)*longitude)] = erroranoms[Imonths15,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360)) #Fill Xmonths15 with training data
        Xmonths20[:,(y*longitude):((y+1)*longitude)] = erroranoms[Imonths20,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360)) #Fill Xmonths20 with training data
        Xmonths20_a[:,(y*longitude):((y+1)*longitude)] = gefsanoms[Imonths20,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360)) #Fill Xmonths20_a with testing data
    Cov = (Xmonths15[Inm_reindex,:].T.dot(Xmonths15[Inm_reindex,:]))/Xmonths15[Inm_reindex,:].shape[0] #Covariance matrix for spatial eigenvectors for training period
    w,v = np.linalg.eigh(Cov) #Calculate monthly eigenvectors and eigenvalues
    w=w.real
    v=v.real
    Isort=np.argsort(-w) #Sorting index numbers
    w=w[Isort]
    v=v[:,Isort]
    W[:,lead]=w
    V[:,:,lead]=v[:,:nretained]
        
        
    w=w/w.sum() #normalize the eigenvalues to create an eigenspectrum
    I80 = np.where(np.cumsum(w)>.8)[0][0] #Retain the EOFs that explain 80% of the variance
    I80s[lead] = I80 #Store retained EOFs according to lead 
    print(I80)
        
    #Create Time Extended EOFs
    for y in np.arange(latitude):
        Xmonths15[:,(y*longitude):((y+1)*longitude)] = erroranoms[Imonths15,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360))
        Xmonths20[:,(y*longitude):((y+1)*longitude)] = erroranoms[Imonths20,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360))
        Xmonths20_a[:,(y*longitude):((y+1)*longitude)] = gefsanoms[Imonths20,y,:]*np.sqrt(np.cos((lats[y]*2*np.pi)/360))
    if lead == 0:
        PC=np.zeros((Xmonths15.shape[0],nretained,nleads))
        PC20_a=np.zeros((Xmonths20_a.shape[0],nretained,nleads))
    PC[:,:,lead] = Xmonths15[:,:].dot(V[:,:nretained,lead]) #Project retainedspatial eigenvectors onto training data according to lead 
    PC20_a[:,:,lead] = Xmonths20_a[:,:].dot(V[:,:nretained,lead]) #Project retainedspatial eigenvectors onto testing data according to lead

        
    print('Finished with spatial eofs, saving eigenvectors and eigen numbers')
    # np.save('/free2/sm957448/200mb_Z/full_eigenvectors/w12_not_std.npy', w)
    # np.save('/free2/sm957448/200mb_Z/full_eigenvectors/v12.npy', v)
    # np.save('/free2/sm957448/200mb_Z/full_eigenvectors/I80s/I80s_12.npy', I80s)
    
    print('Begin time-extended eofs')
#Now we calculate and store the spatial PCs
I80s_cum[1:17] = np.cumsum(I80s) #Store the cumulative sum of the EOFs that explain 80% of the varinace. These will be used to create the time extended data arrays
I80s_cum = I80s_cum.astype(int)
#Create empty arrays for the time extended EOFs to be filled later.
X_ext = np.zeros((Xmonths15.shape[0],int(np.sum(I80s))))
X_ext20_a = np.zeros((Xmonths20_a.shape[0],int(np.sum(I80s))))
    

for lead in np.arange(nleads):    
    X_ext[:,int(I80s_cum[lead]):int(I80s_cum[lead+1])] = PC[:,:int(I80s[lead]),lead] #Fill time extended training array with spatial PCs for time extended EOFs
    X_ext20_a[:,int(I80s_cum[lead]):int(I80s_cum[lead+1])] = PC20_a[:,:int(I80s[lead]),lead] #Fill time extended testing array with spatial PCs for time extended EOFs
            
#Now that we have the time extended X matrices, we restart the EOF process, this time using the time extended matrixes
#Find the time extended covariance matrix:
Cov = (X_ext[Inm_reindex,:].T.dot(X_ext[Inm_reindex,:]))/X_ext[Inm_reindex,:].shape[0] #Define covariance matrix for time extended EOfs

#Find eigenvalues and eigenvectors of Cov
w_ext, v_ext = np.linalg.eigh(Cov) #Calculate time extended eigenvectors and eigenvalues
w_ext = w_ext.real
v_ext = v_ext.real

#Time extended Eigenspectrum (standardizing the eigenvalues)
w_ext = w_ext/w_ext.sum()

#Create index for sorted time extended eigenvalues. Use index when sorting eigenvectors
I = np.argsort(-w_ext)
w_ext = w_ext[I]
v_ext = v_ext[:,I]
# np.save('/pr11/ws347875/200mb_Z/full_eigenvectors/time_extended/Vext_12.npy', v_ext)
# np.save('/pr11/ws347875/200mb_Z/full_eigenvectors/time_extended/Wext_12.npy', w_ext)

print('Done with eofs, begin eigenmode plots')
    
#CREATE EIGENMODE PLOTS

#Define N (previously known as I80) based off of I80s
N = np.zeros(17).astype(int)
N[1:] = np.cumsum(I80s).astype(int)
print(N)
leads=np.arange(nleads)
#Prepare indexing and eigennumbers/vectors
If = np.where(np.cumsum(w_ext)/np.sum(w_ext)>0.8)[0][0] #Index of retained time extended eigenvalues
print('The number of modes retained is '+str(If))
v_ext = v_ext.real
v_ext = v_ext[:,:If] #Define the number of EOFs retained 

errormaps = np.zeros((v.shape[0],nleads,nretained)) #Create empty array that contains the spatial-lead distribution of the leading 200 eigenmodes
for modenum in np.arange(np.min((If,nretained))):
    for lead in np.arange(nleads):
        errormaps[:,lead,modenum]=v[:,:int(I80s[lead])].dot(v_ext[N[lead]:N[lead+1],modenum]) #Fill errormaps according to retained eigenmodes and lead

    errormapslatlon = np.zeros((latitude,longitude,nleads)) #Create empty array that will house errormaps data in a latitude longitude grid
    levs=np.arange(-0.02,0.022,.002) #Define levels for EOF loadings in future contour plot
#    plt.figure(figsize = (9,5))
    # lons = np.arange(0,358,2)
    # lats = np.arange(-89,89,2)
    plt.figure(figsize=(10,7)) #Define Figure

    res = '50m'
    
    print('Modenum = '+str(modenum))
    for y in np.arange(latitude):
        errormapslatlon[y,:,:]=errormaps[y*179:(y+1)*179,:,modenum] #Fill errormapslatlon with data from the leading 6 eigenmodes (3 leading EOF pairs) 
        
    #To plot map convert errormapslatlon to xarray. Define the 2 degree global domain
    # lats = np.arange(-89,89,2)
    # lons = np.arange(0,358,2)
    # lead = np.arange(nleads)
        
    #Convert from numpy to xarray
    errormapslatlon = xr.DataArray(errormapslatlon, coords=[lats, lons, leads], dims=["latitude", "longitude", "lead"])    
        
    #Define the grid to make cyclic point
    # dx = np.abs((lons[2]-lons[0]))
    # dy = np.abs((lats[2]-lats[0]))
    # dx, dy

    # lonRange = np.arange(lonW, lonE + dx, dx)
    # latRange = np.arange(latS, latN + dy, dy)

    #proj_map = ccrs.PlateCarree(central_longitude=0)
        
    #Define regular grid from xarray errormapslatlon
    grid_Reg1 = errormapslatlon.sel(longitude = lonRange, latitude = latRange, method = 'nearest')
    lonsReg, latsReg = grid_Reg1.longitude, grid_Reg1.latitude

#Create Plots for each lead    
    for lead in np.arange(nleads):
        plt.clf()
        ax = plt.subplot(1,1,1, projection = ccrs.PlateCarree())
        ax.set_global()
        ax.coastlines()
        
        #Add Cyclic Point to regular grid by creating adjusted array grid_2d_1 which wraps an additional longitude datapoint on the Prime Meridian
        grid_2d_1, clons = cutil.add_cyclic_point(grid_Reg1.isel(lead = lead), lonsReg)
    
        #Add title
        title1 = str('Mode '+str(modenum+1)+' Lead '+str(lead+1))
        ax.set_title(title1, fontsize = 12)
        
        #Make the plots
        mm = ax.contourf(clons,latsReg, grid_2d_1, levels = levs, extend = 'both', transform = proj_map, cmap = 'bwr')
        ax.set_extent([np.min(lons),np.max(lons), np.min(lats), np.max(lats)], crs = ccrs.PlateCarree())
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels = True, linewidth = 2, color = 'gray', alpha = 0.5, linestyle = '--')
#Save figures in the appropriate path:
        plt.savefig('/free2/sm957448/will_files/200mb_Z/transfer_code_testing/eigenmode_plots/month_'+str(monthi)+'_eof'+str(modenum+1)+'_'+str(lead+1)+'.png')
    print('Done with figures, creating reconstructions')   

#Plots have been created for every lead of the leading 6 eigenmodes for a given month, illustrating the evolution of error patterns with lead. Now we move on to creating the reconstructions of the error anomalies

#CREATE RECONSTRUCTIONS
#Take 80% of the time extended variance (64% of variance for the total dataset)
I80_ext = np.where(np.cumsum(w_ext)>.8)[0][0]

#Create the time extended PCs from the time ext array
PC_ext = X_ext.dot(v_ext[:,:int(I80_ext)]) #Project time extended eigenvectors onto training data to create time-extended PCs
PC_ext20_a = X_ext20_a.dot(v_ext[:,:int(I80_ext)]) #Project time extended eigenvectors onto testing data to create time-extended PCs
    
#Now that we have the time extended EOFs and PCs, we can create the reconstructed error values for projection:
time_ext_recon = PC_ext20_a.dot(v_ext[:,:I80_ext].T) #Define time extended reconstructions
error_recon = np.zeros((Xmonths20_a.shape[0], Xmonths20_a.shape[1],nleads)) #Define spatial reconstructions
for lead in np.arange(nleads): #Create spatial reconstructions by projecting time extended PC based reconstructions onto spatial EOfs
    error_recon[:,:,lead] = time_ext_recon[:,I80s_cum[lead]:I80s_cum[lead+1]].dot(v[:,:int(I80s[lead])].T)

#Convert the spatial reconstructed error array into an array which accounts for latitude and longitude
error_reconll = np.zeros((error_recon.shape[0],gefs_data.shape[1],gefs_data.shape[2],nleads)) #Define spatial reconstructions in latitude and longitude format
for y in np.arange(latitude): #Convert spatial reconstructions into latitude and longitude format
    error_reconll[:,y,:,:] = error_recon[:,y*longitude:(y+1)*longitude,:]
print('Reconstruction for month '+str(monthi)+' complete. Saving reconstruction')
 
#Save the reconstructions for the given month. Code will now move onto the next month
np.save('/free2/sm957448/will_files/200mb_Z/transfer_code_testing/error_reconll_'+str(monthi)+'.npy', error_reconll) #Define globally at the top for all paths

empty gefs_data array created with shape: (7304, 89, 179)
gefs climatology created
working on lead number 0
gefs_data shape
(7304, 89, 179)
gefsanoms created
ERA5 climatology created
ERA5 anomalies created, begin calculating seasonal cycle of error
Total Error Anomalies Created. Beginning Spatial Eigenvector Calculations
12
54
Finished with spatial eofs, saving eigenvectors and eigen numbers
Begin time-extended eofs
working on lead number 1
gefs_data shape
(7304, 89, 179)
gefsanoms created
ERA5 anomalies created, begin calculating seasonal cycle of error
Total Error Anomalies Created. Beginning Spatial Eigenvector Calculations
12
62
Finished with spatial eofs, saving eigenvectors and eigen numbers
Begin time-extended eofs
working on lead number 2
gefs_data shape
(7304, 89, 179)
gefsanoms created
ERA5 anomalies created, begin calculating seasonal cycle of error
Total Error Anomalies Created. Beginning Spatial Eigenvector Calculations
12
74
Finished with spatial eofs, saving eigenvectors

/tmp/ipykernel_22252/2645134982.py:341: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10,7)) #Define Figure


Done with figures, creating reconstructions
Modenum = 21
Done with figures, creating reconstructions
Modenum = 22
Done with figures, creating reconstructions
Modenum = 23
Done with figures, creating reconstructions
Modenum = 24
Done with figures, creating reconstructions
Modenum = 25
Done with figures, creating reconstructions
Modenum = 26
Done with figures, creating reconstructions
Modenum = 27
Done with figures, creating reconstructions
Modenum = 28
Done with figures, creating reconstructions
Modenum = 29
Done with figures, creating reconstructions
Modenum = 30
Done with figures, creating reconstructions
Modenum = 31
Done with figures, creating reconstructions
Modenum = 32
Done with figures, creating reconstructions
Modenum = 33
Done with figures, creating reconstructions
Modenum = 34
Done with figures, creating reconstructions
Modenum = 35
Done with figures, creating reconstructions
Modenum = 36
Done with figures, creating reconstructions
Modenum = 37
Done with figures, creating rec

In [34]:
# Revise to load climatologically data to make climatology plots
import numpy as np
import scipy.io as io
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from datetime import date, timedelta
from datetime import datetime
from scipy import io as io
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import pygrib
import urllib
import ftplib
import os
import xarray as xr
import pandas as pd

from cartopy import crs as ccrs, feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.util as cutil

#SPACIAL DEFININTIONS
latS = -88
latN = 88
lonW = 0
lonE = 356
clons = (lonW+lonE)/2.
nleads = 16
time = np.arange(365)
lats = np.arange(-89,89,2)
lons = np.arange(0,358,2)
lead = np.arange(nleads)
dx = np.abs((lons[2]-lons[0]))
dy = np.abs((lats[2]-lats[0]))
dx, dy
proj_map = ccrs.PlateCarree(central_longitude=0)
lonRange = np.arange(lonW, lonE + dx, dx)
latRange = np.arange(latS, latN + dy, dy)
I = np.arange(365)#Create indexing for one year of data
fig = plt.figure(figsize = (8.5,11))
fig.tight_layout()

ecclimo = np.load('/free2/sm957448/will_files/200mb_Z/ecclimo/ecclimo.npy')
for lead in np.arange(nleads):
    gefsclimo=np.load('/free2/sm957448/will_files/200mb_Z/gefs_climo/gefs_climo_lead'+str(lead)+'.npy') #Loads saved gefs_climo files as a .npy
    #cycle = np.zeros((365,gefsclimo.shape[1],gefsclimo.shape[2])) #Define seasonal cycle array
    
    cycle[I,:,:] = gefsclimo[I,:,:]-ecclimo[I+lead,:,:] #fill seasonal cycle array by subtracting the two climatologies
    if lead == 0: #Seasonal cycle of error does not include initial condition errors. Remove lead 1 errors by subtracting from cycle
        cyc_0 = cycle[I,:,:]
        cycle[I,:,:] == 0
    cycle[I,:,:] = cycle[I,:,:]-cyc_0 #Subtract lead 1 errors
    print('Plotting seasonal cycle of error')

    cycle = xr.DataArray(cycle, coords=[time,lats, lons], dims=["time","latitude", "longitude"])

    #Specify Grids
    grid_Reg1 = cycle.sel(longitude = lonRange, latitude = latRange, method = 'nearest')
    lonsReg, latsReg = grid_Reg1.longitude, grid_Reg1.latitude

    #Define levels in decameters
    cycleLevels = np.arange(-10,11,1)

    #define resolution
    res = '50m'

    day_inc = np.arange(1)
    #PLOT Model Seasonal Cycle of Error
    for day in np.arange(0,365,10):#modify day_inc to find days you would like to assess the seasonal cycle of
        #For increments, replace day_inc in the np.arange with the proper increments
        #for lead in np.arange(nleads):
        plt.clf()
        ax = fig.add_subplot(1,1,1, projection = proj_map)
        ax.add_feature(cfeature.COASTLINE.with_scale(res), lw=1)

        #Add Cyclic Point
        grid_2d_1, clons = cutil.add_cyclic_point(grid_Reg1, lonsReg)
    
        #Add title
        title1 = str('200 hPa Geopotential Height Seasonal Cycle of Error \n DOY = '+str(day)+', Lead = '+str(lead))
        ax.set_title(title1)

        #Contour fills
        SS = ax.contourf(clons, latsReg, grid_2d_1[day,:,:], levels = cycleLevels, cmap = 'bwr', extend = 'both', transform = proj_map)
        cbar = fig.colorbar(SS,shrink=0.5, orientation = 'horizontal', pad = 0.01)
        # fontsize_cbar_tick = 9 
        # cbar.ax.tick_params(labelsize=fontsize_cbar_tick)
        cbar.set_label('Error Anomalies (dam)')
        
        plt.savefig('/free2/sm957448/will_files/200mb_Z/transfer_code_testing/climo_d'+str(day)+'_l'+str(lead)+'.png')
# MODE MONTH LEAD INDEXES
# ANIMATIONS
#PUBLIC HTML

Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error
Plotting seasonal cycle of error


In [41]:
#plt.plot(gefs_data[:,50,50])


#plt.savefig('/free2/sm957448/will_files/200mb_Z/transfer_code/plots/plots.png')
print(np.sum(gefs_data)/ngrids)

282399.1315552432
